<a href="https://colab.research.google.com/github/kuds/reinforce-tactics/blob/main/notebooks/llm_bot_tournament.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 LLM Bot Tournament - Reinforce Tactics

Run interactive tournaments between LLM-powered bots (OpenAI GPT, Claude, Gemini) and built-in rule-based bots (SimpleBot, MediumBot, AdvancedBot)!

**Features:**
- 🎮 Single game runner with detailed turn-by-turn logging
- 🏆 Round-robin tournament system with multiple games per matchup
- 📊 **Elo rating system** for skill-based rankings
- 🗺️ **Multiple maps support** with configurable map pool modes
- 🔑 Flexible API key configuration (environment variables or Google Colab secrets)
- 📈 Comprehensive statistics: wins, losses, draws, win rates, Elo ratings
- 🎯 Customizable model selection (GPT-4o, Claude Sonnet, Gemini Pro, etc.)
- 📝 Conversation logging to debug and analyze LLM reasoning

**Supported Bots:**
- **SimpleBot**: Basic rule-based bot (always available)
- **MediumBot**: Improved rule-based bot with advanced strategies (always available)
- **AdvancedBot**: Most sophisticated rule-based bot (always available)
- **OpenAIBot**: Uses OpenAI GPT models (requires `OPENAI_API_KEY`)
- **ClaudeBot**: Uses Anthropic Claude models (requires `ANTHROPIC_API_KEY`)
- **GeminiBot**: Uses Google Gemini models (requires `GOOGLE_API_KEY`)

**Quick Start:**
1. Install dependencies and clone the repository
2. Configure API keys for the bots you want to use
3. Run a single game or full tournament
4. Analyze the results!

**Estimated API Costs:**
- **GPT-4o-mini**: ~$0.0001-0.0005 per game (recommended for testing)
- **Claude Haiku**: ~$0.0001-0.0003 per game (recommended for testing)
- **Gemini Flash**: Free tier available, ~$0.0001 per game
- **GPT-4o**: ~$0.005-0.02 per game (stronger play, higher cost)
- **Claude Sonnet**: ~$0.003-0.015 per game (stronger play, higher cost)

*Costs vary based on game length and map complexity. Use mini/haiku/flash models for testing!*

## 📦 Setup and Installation

In [ ]:
# Clone the Reinforce Tactics repository if not already present
import os
from pathlib import Path

if not Path('reinforce-tactics').exists():
    print("📥 Cloning Reinforce Tactics repository...")
    !git clone https://github.com/kuds/reinforce-tactics.git
    print("✅ Repository cloned!")
else:
    print("✅ Repository already cloned")

# Change to repository directory
os.chdir('reinforce-tactics')
print(f"\n📂 Current directory: {os.getcwd()}")

In [ ]:
# Install LLM bot dependencies
# These are optional - only install for the bots you plan to use
print("📦 Installing LLM dependencies...\n")

# Install OpenAI (for GPT models)
print("Installing OpenAI...")
!pip install -q openai

# Install Anthropic (for Claude models)
print("Installing Anthropic...")
!pip install -q anthropic

# Install Google Generative AI (for Gemini models)
print("Installing Google Gen AI...")
!pip install -q google-genai

# Install other dependencies if needed
!pip install -q pandas numpy

print("\n✅ All LLM dependencies installed!")

In [ ]:
# Add repository to Python path
import sys
repo_path = os.getcwd()
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)
    print(f"✅ Added to Python path: {repo_path}")
else:
    print(f"✅ Already in Python path: {repo_path}")

In [ ]:
import platform
import torch
from importlib.metadata import version

print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Gymnasium Version: {version('gymnasium')}")
print(f"Numpy Version: {version('numpy')}")
print(f"OpenAI Version: {version('openai')}")
print(f"Anthropic Version: {version('anthropic')}")
print(f"Gemini Version: {version('google.genai')}")

## 🔑 API Key Configuration

You have two options for setting API keys:

### Option 1: Direct Environment Variables (Quick Setup)
Set API keys directly in the cells below. **Note:** These will be visible in the notebook.

### Option 2: Google Colab Secrets (Recommended for Colab)
1. Click the 🔑 key icon in the left sidebar
2. Add secrets with names: `OPENAI_API_KEY`, `ANTHROPIC_API_KEY`, `GOOGLE_API_KEY`
3. Toggle "Notebook access" on for each secret

The code below will check both sources and use Colab secrets if available.

In [ ]:
import os

# Try to use Google Colab secrets first
try:
    from google.colab import userdata
    print("✅ Google Colab detected - checking for secrets...\n")

    # Try to get OpenAI key from secrets
    try:
        openai_key = userdata.get('OPENAI_API_KEY')
        os.environ['OPENAI_API_KEY'] = openai_key
        print("✅ OPENAI_API_KEY loaded from Colab secrets")
    except:
        if 'OPENAI_API_KEY' not in os.environ:
            print("⚠️  OPENAI_API_KEY not found in Colab secrets")

    # Try to get Anthropic key from secrets
    try:
        anthropic_key = userdata.get('ANTHROPIC_API_KEY')
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key
        print("✅ ANTHROPIC_API_KEY loaded from Colab secrets")
    except:
        if 'ANTHROPIC_API_KEY' not in os.environ:
            print("⚠️  ANTHROPIC_API_KEY not found in Colab secrets")

    # Try to get Google key from secrets
    try:
        google_key = userdata.get('GOOGLE_API_KEY')
        os.environ['GOOGLE_API_KEY'] = google_key
        print("✅ GOOGLE_API_KEY loaded from Colab secrets")
    except:
        if 'GOOGLE_API_KEY' not in os.environ:
            print("⚠️  GOOGLE_API_KEY not found in Colab secrets")

except ImportError:
    print("ℹ️  Not running in Google Colab - using environment variables")

# Option 1: Set API keys directly (if not using Colab secrets)
# Uncomment and set your keys below if needed:

# os.environ['OPENAI_API_KEY'] = 'sk-...'
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'
# os.environ['GOOGLE_API_KEY'] = 'AI...'

print("\n" + "="*60)
print("API Key Status:")
print("="*60)
print(f"OpenAI:    {'✅ Configured' if os.environ.get('OPENAI_API_KEY') else '❌ Not set'}")
print(f"Anthropic: {'✅ Configured' if os.environ.get('ANTHROPIC_API_KEY') else '❌ Not set'}")
print(f"Google:    {'✅ Configured' if os.environ.get('GOOGLE_API_KEY') else '❌ Not set'}")
print("="*60)
print("\nℹ️  You can run tournaments with any bots that have API keys configured.")
print("   SimpleBot, MediumBot, and AdvancedBot are always available and don't require API keys.")

## 📚 Import Required Modules

In [ ]:
import logging
from collections import defaultdict
from typing import Dict, List, Any, Optional, Tuple, Union

# Configure logging to see bot actions
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import game components
from reinforcetactics.core.game_state import GameState
from reinforcetactics.game.bot import SimpleBot, MediumBot, AdvancedBot
from reinforcetactics.utils.file_io import FileIO

# Import LLM bots (with graceful fallback)
llm_bots_available = {}

try:
    from reinforcetactics.game.llm_bot import OpenAIBot
    llm_bots_available['openai'] = OpenAIBot
    print("✅ OpenAIBot available")
except ImportError as e:
    print(f"⚠️  OpenAIBot not available: {e}")

try:
    from reinforcetactics.game.llm_bot import ClaudeBot
    llm_bots_available['claude'] = ClaudeBot
    print("✅ ClaudeBot available")
except ImportError as e:
    print(f"⚠️  ClaudeBot not available: {e}")

try:
    from reinforcetactics.game.llm_bot import GeminiBot
    llm_bots_available['gemini'] = GeminiBot
    print("✅ GeminiBot available")
except ImportError as e:
    print(f"⚠️  GeminiBot not available: {e}")

print(f"\n✅ Imports complete! {len(llm_bots_available)} LLM bot types available.")

## 📊 Elo Rating System

The Elo rating system tracks bot performance across games, providing a skill-based ranking.

In [ ]:
class EloRatingSystem:
    """Manages Elo ratings for tournament participants."""

    def __init__(self, starting_elo: int = 1500, k_factor: int = 32):
        """
        Initialize Elo rating system.

        Args:
            starting_elo: Initial Elo rating for all bots (default: 1500)
            k_factor: K-factor for rating changes (default: 32)
        """
        self.starting_elo = starting_elo
        self.k_factor = k_factor
        self.ratings: Dict[str, float] = {}
        self.initial_ratings: Dict[str, float] = {}
        self.rating_history: Dict[str, List[float]] = {}

    def initialize_bot(self, bot_name: str) -> None:
        """Initialize a bot with starting Elo rating."""
        if bot_name not in self.ratings:
            self.ratings[bot_name] = float(self.starting_elo)
            self.initial_ratings[bot_name] = float(self.starting_elo)
            self.rating_history[bot_name] = [float(self.starting_elo)]

    def calculate_expected_score(self, player_elo: float, opponent_elo: float) -> float:
        """Calculate expected score for a player (0.0 to 1.0)."""
        return 1.0 / (1.0 + 10 ** ((opponent_elo - player_elo) / 400.0))

    def update_ratings(self, bot1_name: str, bot2_name: str, result: int) -> None:
        """
        Update Elo ratings after a game.

        Args:
            bot1_name: Name of first bot
            bot2_name: Name of second bot
            result: Game result (1=bot1 wins, 2=bot2 wins, 0=draw)
        """
        self.initialize_bot(bot1_name)
        self.initialize_bot(bot2_name)

        bot1_elo = self.ratings[bot1_name]
        bot2_elo = self.ratings[bot2_name]

        bot1_expected = self.calculate_expected_score(bot1_elo, bot2_elo)
        bot2_expected = self.calculate_expected_score(bot2_elo, bot1_elo)

        if result == 1:  # bot1 wins
            bot1_actual, bot2_actual = 1.0, 0.0
        elif result == 2:  # bot2 wins
            bot1_actual, bot2_actual = 0.0, 1.0
        else:  # draw
            bot1_actual, bot2_actual = 0.5, 0.5

        self.ratings[bot1_name] = bot1_elo + self.k_factor * (bot1_actual - bot1_expected)
        self.ratings[bot2_name] = bot2_elo + self.k_factor * (bot2_actual - bot2_expected)

        self.rating_history[bot1_name].append(self.ratings[bot1_name])
        self.rating_history[bot2_name].append(self.ratings[bot2_name])

    def get_rating(self, bot_name: str) -> float:
        """Get current Elo rating for a bot."""
        return self.ratings.get(bot_name, float(self.starting_elo))

    def get_rating_change(self, bot_name: str) -> float:
        """Get Elo rating change since tournament start."""
        initial = self.initial_ratings.get(bot_name, float(self.starting_elo))
        current = self.ratings.get(bot_name, float(self.starting_elo))
        return current - initial

print("✅ EloRatingSystem class defined")

## 🎮 Single Game Runner

Run a single game between two bots with detailed logging.

In [ ]:
def run_single_game(
    player1_bot: Union[str, type],
    player2_bot: Union[str, type],
    map_file: str = 'maps/1v1/6x6_beginner.csv',
    max_turns: int = 500,
    verbose: bool = True,
    player1_model: Optional[str] = None,
    player2_model: Optional[str] = None,
    player1_temperature: Optional[float] = None,
    player2_temperature: Optional[float] = None,
    player1_max_tokens: int = 8000,
    player2_max_tokens: int = 8000,
    player1_should_reason: bool = False,
    player2_should_reason: bool = False,
    log_conversations: bool = False,
    conversation_log_dir: Optional[str] = None,
    save_replay: bool = False,
    replay_dir: Optional[str] = None
) -> int:
    """
    Run a single game between two bots.

    Args:
        player1_bot: Bot class or 'simple'/'medium'/'advanced' for built-in bots (plays as Player 1)
        player2_bot: Bot class or 'simple'/'medium'/'advanced' for built-in bots (plays as Player 2)
        map_file: Path to map file (default: maps/1v1/6x6_beginner.csv)
        max_turns: Maximum number of turns to prevent infinite games (default: 500)
        verbose: Show turn-by-turn details (default: True)
        player1_model: Optional model name for player 1 LLM bot
        player2_model: Optional model name for player 2 LLM bot
        player1_temperature: Optional temperature for player 1 LLM bot (0.0 to 1.0)
        player2_temperature: Optional temperature for player 2 LLM bot (0.0 to 1.0)
        player1_max_tokens: Max output tokens for player 1 LLM bot (default: 8000)
        player2_max_tokens: Max output tokens for player 2 LLM bot (default: 8000)
        player1_should_reason: Include reasoning field in player 1 LLM response format (default: False)
        player2_should_reason: Include reasoning field in player 2 LLM response format (default: False)
        log_conversations: Enable conversation logging for LLM bots (default: False)
        conversation_log_dir: Directory for conversation logs (optional)
        save_replay: Whether to save the game replay (default: False)
        replay_dir: Directory for replay files (default: llm_replays/)

    Returns:
        Winner: 1 (player 1 wins), 2 (player 2 wins), or 0 (draw)
    """
    # Load map
    map_data = FileIO.load_map(map_file)
    if map_data is None:
        raise ValueError(f"Failed to load map: {map_file}")

    # Create game state
    game_state = GameState(map_data, num_players=2)

    # Store map file reference for replay (required for replay system to load map)
    game_state.map_file_used = map_file

    # Handle conversation log directory
    abs_log_dir = None
    log_file_count_before = 0
    if log_conversations and conversation_log_dir:
        # Convert to absolute path
        abs_log_dir = os.path.abspath(conversation_log_dir)
        # Create directory if it doesn't exist
        try:
            os.makedirs(abs_log_dir, exist_ok=True)
            if verbose:
                print(f"📁 Log directory created/verified: {abs_log_dir}")
        except Exception as e:
            print(f"⚠️  Warning: Could not create log directory: {e}")
            abs_log_dir = None

        # Count existing log files
        if abs_log_dir and os.path.exists(abs_log_dir):
            log_file_count_before = len([f for f in os.listdir(abs_log_dir) if f.endswith('.json')])

    # Handle replay directory
    abs_replay_dir = None
    if save_replay:
        # Use default directory if not specified
        if replay_dir is None:
            replay_dir = 'llm_replays'
        # Convert to absolute path
        abs_replay_dir = os.path.abspath(replay_dir)
        # Create directory if it doesn't exist
        try:
            os.makedirs(abs_replay_dir, exist_ok=True)
            if verbose:
                print(f"📁 Replay directory created/verified: {abs_replay_dir}")
        except Exception as e:
            print(f"⚠️  Warning: Could not create replay directory: {e}")
            abs_replay_dir = None
            save_replay = False

    # Create bot instances
    def create_bot(bot_spec, player_num, model=None, temperature=None, max_tokens=None, should_reason=False):
        if bot_spec == 'simple' or bot_spec is None:
            return SimpleBot(game_state, player_num)
        elif bot_spec == 'medium':
            return MediumBot(game_state, player_num)
        elif bot_spec == 'advanced':
            return AdvancedBot(game_state, player_num)
        else:
            # It's an LLM bot class
            kwargs = {'game_state': game_state, 'player': player_num}
            if model:
                kwargs['model'] = model
            if temperature is not None:
                kwargs['temperature'] = temperature
            if max_tokens is not None:
                kwargs['max_tokens'] = max_tokens
            kwargs['should_reason'] = should_reason
            if log_conversations:
                kwargs['log_conversations'] = log_conversations
            if abs_log_dir:
                kwargs['conversation_log_dir'] = abs_log_dir
            return bot_spec(**kwargs)

    bot1 = create_bot(player1_bot, 1, player1_model, player1_temperature, player1_max_tokens, player1_should_reason)
    bot2 = create_bot(player2_bot, 2, player2_model, player2_temperature, player2_max_tokens, player2_should_reason)
    bots = {1: bot1, 2: bot2}

    # Get bot names
    bot1_name = bot1.__class__.__name__
    bot2_name = bot2.__class__.__name__

    if verbose:
        print("\n" + "="*60)
        print(f"Game Start: {bot1_name} (P1) vs {bot2_name} (P2)")
        print(f"Map: {map_file}")
        if player1_should_reason or player2_should_reason:
            reasoning_info = []
            if player1_should_reason:
                reasoning_info.append("P1")
            if player2_should_reason:
                reasoning_info.append("P2")
            print(f"Reasoning Enabled: {', '.join(reasoning_info)}")
        if player1_temperature is not None or player2_temperature is not None:
            temp_info = []
            if player1_temperature is not None: temp_info.append(f"P1={player1_temperature}")
            if player2_temperature is not None: temp_info.append(f"P2={player2_temperature}")
            print(f"Temperature: {', '.join(temp_info)}")
        if log_conversations:
            print(f"Conversation Logging: ENABLED")
            if abs_log_dir:
                print(f"📁 Absolute Log Path: {abs_log_dir}")
        if save_replay:
            print(f"Replay Saving: ENABLED")
            if abs_replay_dir:
                print(f"🎬 Replay Directory: {abs_replay_dir}")
        print("="*60)

    # Play the game
    turn_count = 0
    last_gold = {1: game_state.player_gold[1], 2: game_state.player_gold[2]}

    # Import time for retry delays
    import time

    while not game_state.game_over and turn_count < max_turns:
        current_player = game_state.current_player
        current_bot = bots[current_player]
        bot_name = bot1_name if current_player == 1 else bot2_name

        # Identify if current bot is an LLM bot
        is_llm_bot = current_bot.__class__.__name__ in ['OpenAIBot', 'ClaudeBot', 'GeminiBot']

        # Show turn info
        if verbose:
            print(f"\n--- Turn {turn_count + 1} - {bot_name} (P{current_player}) ---")
            print(f"  Gold: P1={game_state.player_gold[1]}, P2={game_state.player_gold[2]}")

        if is_llm_bot:
            # Bot takes turn with retry logic for API limits (ONLY for LLM bots)
            max_retries = 3
            retry_delay = 5

            for attempt in range(max_retries + 1):
                try:
                    current_bot.take_turn()
                    # ALWAYS wait 0.5s for LLM bots
                    time.sleep(0.5)
                    break # Success
                except Exception as e:
                    error_str = str(e)
                    # Check for rate limit errors (429) or overloaded errors (503/529)
                    is_rate_limit = "429" in error_str or "Too Many Requests" in error_str or "Overloaded" in error_str

                    if is_rate_limit and attempt < max_retries:
                        wait_time = retry_delay * (2 ** attempt) # Exponential backoff
                        if verbose:
                            print(f"⚠️  Rate limit/API error during {bot_name} turn: {e}")
                            print(f"⏳ Waiting {wait_time}s before retry ({attempt+1}/{max_retries})...")
                        else:
                            print(f"⏳ Rate limit hit. Waiting {wait_time}s...")
                        time.sleep(wait_time)
                    else:
                        # Final failure or non-retriable error
                        print(f"⚠️  Fatal error during {bot_name} turn: {e}")
                        # Bot forfeits on error
                        game_state.game_over = True
                        game_state.winner = 1 if current_player == 2 else 2
                        break
        else:
            # Regular bot take turn (no retries, no forced delays)
            try:
                current_bot.take_turn()
            except Exception as e:
                print(f"⚠️  Fatal error during {bot_name} turn: {e}")
                game_state.game_over = True
                game_state.winner = 1 if current_player == 2 else 2

        # Show gold changes
        if verbose and not game_state.game_over:
            gold_change = game_state.player_gold[current_player] - last_gold[current_player]
            if gold_change != 0:
                print(f"  Gold change: {gold_change:+d}")
            last_gold[current_player] = game_state.player_gold[current_player]

        turn_count += 1

        # Check for game over
        if game_state.game_over:
            break

    # Determine winner
    if game_state.game_over and game_state.winner:
        winner = game_state.winner
        winner_name = bot1_name if winner == 1 else bot2_name
    elif turn_count >= max_turns:
        winner = 0
        winner_name = "Draw (max turns)"
    else:
        winner = 0
        winner_name = "Draw"

    if verbose:
        print("\n" + "="*60)
        print(f"Game Over! Winner: {winner_name}")
        print(f"Total turns: {turn_count}")
        print(f"Final gold - P1: {game_state.player_gold[1]}, P2: {game_state.player_gold[2]}")
        print("="*60 + "\n")

    # Save replay if enabled
    if save_replay and abs_replay_dir:
        from datetime import datetime
        from pathlib import Path

        # Generate descriptive filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Build bot descriptions for filename
        def get_bot_desc(bot_name, model):
            if model:
                # Clean model name for filename (sanitize to alphanumeric, dash, underscore)
                import re
                clean_model = re.sub(r'[^\w\-]', '-', model)
                return f"{bot_name}-{clean_model}"
            return bot_name

        bot1_desc = get_bot_desc(bot1_name, player1_model)
        bot2_desc = get_bot_desc(bot2_name, player2_model)

        # Winner description
        if winner == 1:
            winner_desc = "P1wins"
        elif winner == 2:
            winner_desc = "P2wins"
        else:
            winner_desc = "Draw"

        replay_filename = f"game_{timestamp}_{bot1_desc}_vs_{bot2_desc}_{winner_desc}.json"
        replay_path = Path(abs_replay_dir) / replay_filename

        try:
            saved_path = game_state.save_replay_to_file(str(replay_path))
            if saved_path and verbose:
                print(f"🎬 Replay saved: {replay_path}")
        except Exception as e:
            print(f"⚠️  Error saving replay: {e}")

    # --- Force Save and Cleanup for Logs ---
    # Ensure bots save their logs before we check for files
    if log_conversations:
        for bot in bots.values():
            # Try common save method names if they exist
            if hasattr(bot, 'save_conversation_log'):
                try:
                    bot.save_conversation_log()
                except Exception as e:
                    print(f"⚠️ Error saving bot logs: {e}")

    # Force cleanup of bot instances to trigger any __del__ log saving
    del bot1
    del bot2
    del bots
    import gc
    gc.collect()
    # ----------------------------------------

    # Show log file summary
    if log_conversations and abs_log_dir and os.path.exists(abs_log_dir):
        log_files = [f for f in os.listdir(abs_log_dir) if f.endswith('.json')]
        new_log_count = len(log_files) - log_file_count_before

        # If verbose is OFF, we still want a small indicator if logs were saved
        if new_log_count > 0:
            if verbose:
                print("\n" + "="*60)
                print("📊 Conversation Logs Summary")
                print("="*60)
                print(f"✅ Generated {new_log_count} new log file(s)")
                print(f"📁 Absolute path: {abs_log_dir}")
                print(f"📝 Total log files in directory: {len(log_files)}")
                print(f"\n💡 To list log files, run:")
                print(f"   !ls -lh {abs_log_dir}")
                print("="*60 + "\n")
            else:
                # Minimal output for tournaments
                print(f"    📝 +{new_log_count} log(s) saved")

    return winner

## 🏆 Tournament Runner

Run a round-robin tournament between multiple bots.

In [ ]:
class TournamentBot:
    """Configuration for a bot participating in a tournament."""
    def __init__(self, name: str, bot_class: Union[str, type], model: Optional[str] = None,
                 temperature: Optional[float] = None, max_tokens: int = 8000):
        self.name = name
        self.bot_class = bot_class
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens

def run_tournament(
    bots: List[TournamentBot],
    map_file: str = 'maps/1v1/6x6_beginner.csv',
    maps: Optional[List[str]] = None,
    map_pool_mode: str = 'all',
    games_per_matchup: int = 2,
    max_turns: int = 500,
    should_reason: bool = False,
    log_conversations: bool = False,
    conversation_log_dir: Optional[str] = None,
    save_replays: bool = False,
    replay_dir: Optional[str] = None
) -> Dict[str, Any]:
    """
    Run a round-robin tournament between multiple bots with Elo ratings.

    Args:
        bots: List of TournamentBot objects describing each bot.
        map_file: Path to single map file (default: maps/1v1/6x6_beginner.csv)
        maps: List of map file paths to use (takes precedence over map_file)
        map_pool_mode: How to select maps - 'all' (play every map), 'cycle', or 'random'
        games_per_matchup: Games per side per map (total = 2 * games_per_matchup * num_maps if 'all')
        max_turns: Maximum turns per game (default: 500)
        should_reason: Include reasoning field in LLM response format for all LLM bots (default: False).
            When True, LLM bots will explain their strategic reasoning in each response.
        log_conversations: Enable conversation logging for LLM bots (default: False)
        conversation_log_dir: Directory for conversation logs (optional)
        save_replays: Whether to save replays for all games (default: False)
        replay_dir: Directory for replay files (default: llm_replays/)

    Returns:
        Dictionary with tournament results, standings, and Elo ratings
    """
    import time

    if len(bots) < 2:
        raise ValueError("Need at least 2 bots for a tournament")

    # Handle map list
    if maps:
        map_list = maps
    else:
        map_list = [map_file]

    # Initialize Elo rating system
    elo_system = EloRatingSystem()
    for bot in bots:
        elo_system.initialize_bot(bot.name)

    # Handle conversation log directory
    abs_log_dir = None
    initial_log_count = 0
    if log_conversations and conversation_log_dir:
        # Convert to absolute path
        abs_log_dir = os.path.abspath(conversation_log_dir)
        # Create directory if it doesn't exist
        try:
            os.makedirs(abs_log_dir, exist_ok=True)
            print(f"📁 Log directory created/verified: {abs_log_dir}\n")
        except Exception as e:
            print(f"⚠️  Warning: Could not create log directory: {e}")
            abs_log_dir = None

        # Count existing log files (recursively)
        if abs_log_dir and os.path.exists(abs_log_dir):
            for root, dirs, files in os.walk(abs_log_dir):
                initial_log_count += len([f for f in files if f.endswith('.json')])

    # Handle replay directory
    abs_replay_dir = None
    initial_replay_count = 0
    if save_replays:
        # Use default directory if not specified
        if replay_dir is None:
            replay_dir = 'llm_replays'
        # Convert to absolute path
        abs_replay_dir = os.path.abspath(replay_dir)
        # Create directory if it doesn't exist
        try:
            os.makedirs(abs_replay_dir, exist_ok=True)
            print(f"🎬 Replay directory created/verified: {abs_replay_dir}\n")
        except Exception as e:
            print(f"⚠️  Warning: Could not create replay directory: {e}")
            abs_replay_dir = None
            save_replays = False

        # Count existing replay files (recursively)
        if abs_replay_dir and os.path.exists(abs_replay_dir):
            for root, dirs, files in os.walk(abs_replay_dir):
                initial_replay_count += len([f for f in files if f.endswith('.json')])

    print("\n" + "="*70)
    print(f"🏆 TOURNAMENT START")
    print("="*70)
    if len(map_list) == 1:
        print(f"Map: {map_list[0]}")
    else:
        print(f"Maps: {len(map_list)} maps")
        for m in map_list:
            print(f"  - {m}")
        print(f"Map Pool Mode: {map_pool_mode}")
    print(f"Participants: {len(bots)}")

    for bot in bots:
        model_str = f" ({bot.model})" if bot.model else ""
        temp_str = f" [temp={bot.temperature}]" if bot.temperature is not None else ""

        if bot.bot_class == 'simple':
            bot_type_str = "SimpleBot"
        elif bot.bot_class == 'medium':
            bot_type_str = "MediumBot"
        elif bot.bot_class == 'advanced':
            bot_type_str = "AdvancedBot"
        else:
            bot_type_str = bot.bot_class.__name__
        print(f"  - {bot.name}: {bot_type_str}{model_str}{temp_str}")

    # Calculate total games
    if map_pool_mode == 'all' and len(map_list) > 1:
        games_per_matchup_total = games_per_matchup * 2 * len(map_list)
    else:
        games_per_matchup_total = games_per_matchup * 2

    print(f"Games per matchup: {games_per_matchup_total}")
    if should_reason:
        print(f"LLM Reasoning: ENABLED (bots will explain their strategy)")
    if log_conversations:
        print(f"LLM Conversation Logging: ENABLED")
        if abs_log_dir:
            print(f"📁 Absolute Log Path: {abs_log_dir}")
            print(f"📂 Logs will be grouped by map/matchup")
    if save_replays:
        print(f"Replay Saving: ENABLED")
        if abs_replay_dir:
            print(f"🎬 Replay Directory: {abs_replay_dir}")
            print(f"📂 Replays will be grouped by map/matchup")
    print("="*70 + "\n")

    # Initialize results tracking
    results = defaultdict(lambda: {'wins': 0, 'losses': 0, 'draws': 0})
    per_map_stats = defaultdict(lambda: defaultdict(lambda: {'wins': 0, 'losses': 0, 'draws': 0}))
    matchup_details = []
    current_map_index = 0

    # Helper function to select map
    def select_map(game_num):
        nonlocal current_map_index
        if len(map_list) == 1:
            return map_list[0]
        if map_pool_mode == 'cycle':
            selected = map_list[current_map_index % len(map_list)]
            current_map_index += 1
            return selected
        elif map_pool_mode == 'random':
            import random
            return random.choice(map_list)
        else:  # 'all' mode - handled separately
            return map_list[(game_num - 1) % len(map_list)]

    # Generate all matchups (round-robin)
    matchups = []
    for i in range(len(bots)):
        for j in range(i + 1, len(bots)):
            matchups.append((i, j))

    total_games = len(matchups) * games_per_matchup_total
    print(f"📊 Total matchups: {len(matchups)}")
    print(f"📊 Total games: {total_games}\n")

    game_num = 0

    # Known LLM bot class names for detecting if we should log/retry
    known_llm_bots = ['OpenAIBot', 'ClaudeBot', 'GeminiBot']

    # Run all matchups
    for matchup_idx, (i, j) in enumerate(matchups, 1):
        bot1 = bots[i]
        bot2 = bots[j]

        print(f"\n{'='*70}")
        print(f"Matchup {matchup_idx}/{len(matchups)}: {bot1.name} vs {bot2.name}")
        print(f"{'='*70}")

        # Check if matchup involves LLM bots to decide on logging/delays
        has_llm_bot = False
        bot1_is_llm = False
        bot2_is_llm = False
        if not isinstance(bot1.bot_class, str) and bot1.bot_class.__name__ in known_llm_bots:
            has_llm_bot = True
            bot1_is_llm = True
        if not isinstance(bot2.bot_class, str) and bot2.bot_class.__name__ in known_llm_bots:
            has_llm_bot = True
            bot2_is_llm = True

        matchup_results = {
            'bot1': bot1.name,
            'bot2': bot2.name,
            'bot1_wins': 0,
            'bot2_wins': 0,
            'draws': 0
        }

        # Determine which maps to play
        if map_pool_mode == 'all' and len(map_list) > 1:
            # Play all maps for each side
            maps_to_play = map_list
        else:
            # Single map or cycle/random mode
            maps_to_play = [None]  # Will select dynamically

        for current_map in maps_to_play:
            # Define matchup folder name (consistent for both sides)
            matchup_folder = f"{bot1.name}_vs_{bot2.name}"

            # Play games_per_matchup with bot1 as player 1
            for game in range(games_per_matchup):
                game_num += 1
                selected_map = current_map if current_map else select_map(game_num)
                map_name = os.path.basename(selected_map)
                print(f"\n  Game {game_num}/{total_games}: {bot1.name} (P1) vs {bot2.name} (P2) on {map_name}")

                # Determine subdirectory for map and matchup
                map_subdir = os.path.join(os.path.splitext(map_name)[0], matchup_folder)

                # Configure specific directories
                game_log_dir = None
                # Only create conversation log dir if enabled AND we have LLM bots
                if log_conversations and has_llm_bot and (abs_log_dir or conversation_log_dir):
                    base_log = abs_log_dir if abs_log_dir else conversation_log_dir
                    game_log_dir = os.path.join(base_log, map_subdir)

                game_replay_dir = None
                if save_replays and (abs_replay_dir or replay_dir):
                    base_replay = abs_replay_dir if abs_replay_dir else replay_dir
                    game_replay_dir = os.path.join(base_replay, map_subdir)

                winner = run_single_game(
                    bot1.bot_class, bot2.bot_class,
                    map_file=selected_map,
                    max_turns=max_turns,
                    verbose=False,
                    player1_model=bot1.model,
                    player2_model=bot2.model,
                    player1_temperature=bot1.temperature,
                    player2_temperature=bot2.temperature,
                    player1_max_tokens=bot1.max_tokens,
                    player2_max_tokens=bot2.max_tokens,
                    player1_should_reason=should_reason if bot1_is_llm else False,
                    player2_should_reason=should_reason if bot2_is_llm else False,
                    log_conversations=log_conversations if has_llm_bot else False,
                    conversation_log_dir=game_log_dir,
                    save_replay=save_replays,
                    replay_dir=game_replay_dir
                )

                if winner == 1:
                    results[bot1.name]['wins'] += 1
                    results[bot2.name]['losses'] += 1
                    matchup_results['bot1_wins'] += 1
                    per_map_stats[bot1.name][map_name]['wins'] += 1
                    per_map_stats[bot2.name][map_name]['losses'] += 1
                    elo_system.update_ratings(bot1.name, bot2.name, 1)
                    print(f"    ✅ {bot1.name} wins!")
                elif winner == 2:
                    results[bot2.name]['wins'] += 1
                    results[bot1.name]['losses'] += 1
                    matchup_results['bot2_wins'] += 1
                    per_map_stats[bot2.name][map_name]['wins'] += 1
                    per_map_stats[bot1.name][map_name]['losses'] += 1
                    elo_system.update_ratings(bot1.name, bot2.name, 2)
                    print(f"    ✅ {bot2.name} wins!")
                else:
                    results[bot1.name]['draws'] += 1
                    results[bot2.name]['draws'] += 1
                    matchup_results['draws'] += 1
                    per_map_stats[bot1.name][map_name]['draws'] += 1
                    per_map_stats[bot2.name][map_name]['draws'] += 1
                    elo_system.update_ratings(bot1.name, bot2.name, 0)
                    print(f"    ⚖️  Draw")

                # Small delay to prevent rate limits (only for LLM matches)
                if has_llm_bot:
                    time.sleep(2)

            # Play games_per_matchup with bot2 as player 1 (swap sides)
            for game in range(games_per_matchup):
                game_num += 1
                selected_map = current_map if current_map else select_map(game_num)
                map_name = os.path.basename(selected_map)
                print(f"\n  Game {game_num}/{total_games}: {bot2.name} (P1) vs {bot1.name} (P2) on {map_name}")

                # Determine subdirectory for map and matchup
                # Note: Using consistent matchup folder name from outer loop logic
                map_subdir = os.path.join(os.path.splitext(map_name)[0], matchup_folder)

                # Configure specific directories
                game_log_dir = None
                # Only create conversation log dir if enabled AND we have LLM bots
                if log_conversations and has_llm_bot and (abs_log_dir or conversation_log_dir):
                    base_log = abs_log_dir if abs_log_dir else conversation_log_dir
                    game_log_dir = os.path.join(base_log, map_subdir)

                game_replay_dir = None
                if save_replays and (abs_replay_dir or replay_dir):
                    base_replay = abs_replay_dir if abs_replay_dir else replay_dir
                    game_replay_dir = os.path.join(base_replay, map_subdir)

                winner = run_single_game(
                    bot2.bot_class, bot1.bot_class,
                    map_file=selected_map,
                    max_turns=max_turns,
                    verbose=False,
                    player1_model=bot2.model,
                    player2_model=bot1.model,
                    player1_temperature=bot2.temperature,
                    player2_temperature=bot1.temperature,
                    player1_max_tokens=bot2.max_tokens,
                    player2_max_tokens=bot1.max_tokens,
                    player1_should_reason=should_reason if bot2_is_llm else False,
                    player2_should_reason=should_reason if bot1_is_llm else False,
                    log_conversations=log_conversations if has_llm_bot else False,
                    conversation_log_dir=game_log_dir,
                    save_replay=save_replays,
                    replay_dir=game_replay_dir
                )

                if winner == 1:
                    results[bot2.name]['wins'] += 1
                    results[bot1.name]['losses'] += 1
                    matchup_results['bot2_wins'] += 1
                    per_map_stats[bot2.name][map_name]['wins'] += 1
                    per_map_stats[bot1.name][map_name]['losses'] += 1
                    elo_system.update_ratings(bot2.name, bot1.name, 1)
                    print(f"    ✅ {bot2.name} wins!")
                elif winner == 2:
                    results[bot1.name]['wins'] += 1
                    results[bot2.name]['losses'] += 1
                    matchup_results['bot1_wins'] += 1
                    per_map_stats[bot1.name][map_name]['wins'] += 1
                    per_map_stats[bot2.name][map_name]['losses'] += 1
                    elo_system.update_ratings(bot2.name, bot1.name, 2)
                    print(f"    ✅ {bot1.name} wins!")
                else:
                    results[bot1.name]['draws'] += 1
                    results[bot2.name]['draws'] += 1
                    matchup_results['draws'] += 1
                    per_map_stats[bot1.name][map_name]['draws'] += 1
                    per_map_stats[bot2.name][map_name]['draws'] += 1
                    elo_system.update_ratings(bot2.name, bot1.name, 0)
                    print(f"    ⚖️  Draw")

                # Small delay to prevent rate limits (only for LLM matches)
                if has_llm_bot:
                    time.sleep(2)

        # Show matchup summary
        print(f"\n  Matchup Summary: {bot1.name} {matchup_results['bot1_wins']}-{matchup_results['bot2_wins']}-{matchup_results['draws']} {bot2.name}")
        matchup_details.append(matchup_results)

    # Calculate final standings with Elo
    standings = []
    for bot_name, stats in results.items():
        total_games_played = stats['wins'] + stats['losses'] + stats['draws']
        win_rate = stats['wins'] / total_games_played if total_games_played > 0 else 0.0
        elo = elo_system.get_rating(bot_name)
        elo_change = elo_system.get_rating_change(bot_name)

        standings.append({
            'name': bot_name,
            'wins': stats['wins'],
            'losses': stats['losses'],
            'draws': stats['draws'],
            'total': total_games_played,
            'win_rate': win_rate,
            'elo': round(elo, 0),
            'elo_change': round(elo_change, 0)
        })

    # Sort by Elo rating (descending)
    standings.sort(key=lambda x: x['elo'], reverse=True)

    # Display final standings
    print("\n\n" + "="*85)
    print("🏆 FINAL STANDINGS")
    print("="*85)
    print(f"{'Rank':<6}{'Bot':<20}{'Wins':<8}{'Losses':<8}{'Draws':<8}{'Win Rate':<10}{'Elo':<8}{'Δ Elo':<8}")
    print("-"*85)

    for rank, standing in enumerate(standings, 1):
        medal = "🥇" if rank == 1 else ("🥈" if rank == 2 else ("🥉" if rank == 3 else "  "))
        elo_change_str = f"{standing['elo_change']:+.0f}"
        print(f"{medal} {rank:<3}{standing['name']:<20}{standing['wins']:<8}{standing['losses']:<8}"
              f"{standing['draws']:<8}{standing['win_rate']:.3f}{'':3}{standing['elo']:<8.0f}{elo_change_str:<8}")

    print("="*85 + "\n")

    # Show per-map performance if multiple maps were used
    if len(map_list) > 1:
        print("="*85)
        print("📈 Per-Map Performance")
        print("="*85)
        for m in map_list:
            map_name = os.path.basename(m)
            print(f"\n🗺️  {map_name}")
            # Find best performer on this map
            map_wins = {}
            for bot_name in per_map_stats:
                if map_name in per_map_stats[bot_name]:
                    map_wins[bot_name] = per_map_stats[bot_name][map_name]['wins']
            if map_wins:
                best_bot = max(map_wins.items(), key=lambda x: x[1])[0]
                print(f"   Best Performer: {best_bot}")
        print("\n" + "="*85 + "\n")

    # Show replay file summary
    if save_replays and abs_replay_dir and os.path.exists(abs_replay_dir):
        # Count new files recursively
        current_replay_count = 0
        for root, dirs, files in os.walk(abs_replay_dir):
            current_replay_count += len([f for f in files if f.endswith('.json')])

        new_replay_count = current_replay_count - initial_replay_count

        print("="*70)
        print("🎬 Tournament Replays Summary")
        print("="*70)
        print(f"✅ Total games played: {total_games}")
        print(f"🎬 New replay files generated: {new_replay_count}")
        print(f"📁 Absolute path: {abs_replay_dir}")
        print(f"📂 Organized in subfolders by map/matchup")
        print(f"📄 Total replay files in directory: {current_replay_count}")
        print(f"\n💡 To list replay files, run:")
        print(f"   !find {abs_replay_dir} -name '*.json'")
        print(f"\n💡 To view a replay, use the replay player in the game UI")
        print("="*70 + "\n")

    # Show log file summary
    if log_conversations and abs_log_dir and os.path.exists(abs_log_dir):
        # Count new files recursively
        current_log_count = 0
        for root, dirs, files in os.walk(abs_log_dir):
            current_log_count += len([f for f in files if f.endswith('.json')])

        new_log_count = current_log_count - initial_log_count

        print("="*70)
        print("📊 Tournament Logs Summary")
        print("="*70)
        print(f"✅ Total games played: {total_games}")
        print(f"📝 New log files generated: {new_log_count}")
        print(f"📁 Absolute path: {abs_log_dir}")
        print(f"📂 Organized in subfolders by map/matchup")
        print(f"📄 Total log files in directory: {current_log_count}")
        print(f"\n💡 To list log files, run:")
        print(f"   !find {abs_log_dir} -name '*.json'")
        print(f"\n💡 To review logs, see Example 2.6")
        print("="*70 + "\n")

    return {
        'standings': standings,
        'matchups': matchup_details,
        'maps_used': map_list,
        'map_pool_mode': map_pool_mode,
        'games_per_matchup': games_per_matchup,
        'should_reason': should_reason,
        'elo_history': {bot_name: [round(r, 0) for r in history]
                       for bot_name, history in elo_system.rating_history.items()},
        'per_map_stats': {bot_name: dict(map_stats)
                         for bot_name, map_stats in per_map_stats.items()}
    }

### Example 1: Single Game - SimpleBot vs SimpleBot

Let's start with games between built-in bots to test the system. You can use 'simple', 'medium', or 'advanced' as bot types.

### 🎬 Replay Saving

Both `run_single_game()` and `run_tournament()` now support saving game replays!

**Features:**
- 📼 Save replays to view later with the game's replay player
- 📁 Customizable replay directory (default: `llm_replays/`)
- 🏷️ Descriptive filenames with timestamps, bot names, models, and game results
- 🗺️ Replays include map reference for proper playback

**Usage Examples:**

```python
# Single game with replay
winner = run_single_game(
    player1_bot='simple',
    player2_bot='simple',
    save_replay=True,
    replay_dir='my_replays'  # Optional, defaults to 'llm_replays/'
)

# Tournament with replays
results = run_tournament(
    bots=[
        ('SimpleBot', 'simple', None),
        ('GPT-4o-mini', OpenAIBot, 'gpt-4o-mini')
    ],
    save_replays=True,
    replay_dir='tournament_replays'
)
```

**Replay Filename Format:**
```
game_20251217_014426_ClaudeBot-claude-3-haiku_vs_SimpleBot_P1wins.json
```

**Viewing Replays:**
- Use the game's built-in replay player UI
- Replays are saved as JSON files that can be loaded and viewed
- Each replay includes the full game state and action history


In [ ]:
# Example: Run a single game with replay saving
winner = run_single_game(
    player1_bot='simple',
    player2_bot='simple',
    map_file='maps/1v1/corner_points.csv',
    max_turns=10,
    verbose=True,
    save_replay=True,
    replay_dir='llm_replays'  # Optional: defaults to 'llm_replays/'
)

print(f"\nWinner: Player {winner}" if winner else "\nResult: Draw")
print("\n💡 Check the 'llm_replays/' directory for the saved replay file!")

# Try different bot types:
# winner = run_single_game(player1_bot='medium', player2_bot='simple', ...)
# winner = run_single_game(player1_bot='advanced', player2_bot='medium', ...)


### Example 2: Single Game - LLM Bot vs Built-in Bots

Test an LLM bot against built-in bots (SimpleBot, MediumBot, or AdvancedBot). Make sure you have the appropriate API key configured!

In [ ]:
# Run a single game: LLM Bot vs SimpleBot

# Configuration: Choose 'gemini', 'openai', or 'claude'
llm_provider = 'gemini'

# Set logging level
logging.getLogger('reinforcetactics.game.llm_bot').setLevel(logging.DEBUG)

if llm_provider in llm_bots_available:
    # Select appropriate model based on provider
    if llm_provider == 'openai':
        model_name = 'gpt-5-mini-2025-08-07'
    elif llm_provider == 'gemini':
        model_name = 'gemini-2.0-flash'
    elif llm_provider == 'claude':
        model_name = 'claude-haiku-4-5-20251001'
    else:
        model_name = None

    print(f"🎮 Starting game with {llm_provider} bot (model: {model_name})...")

    winner = run_single_game(
        player1_bot=llm_bots_available[llm_provider],
        player2_bot='simple',
        map_file='maps/1v1/beginner.csv',
        max_turns=20,
        verbose=True,
        player1_temperature=0.5,
        player1_model=model_name,
        player1_should_reason=True,  # Enable reasoning to see LLM's strategy
        log_conversations=True,
        save_replay=True,
        conversation_log_dir="llm_logs"
    )
    print(f"\nWinner: Player {winner}" if winner else "\nResult: Draw")
else:
    print(f"⚠️  {llm_provider} bot not available. Please install necessary package and configure API key.")

### Example 2.5: Single Game with Conversation Logging

Enable conversation logging to see the LLM's reasoning process. Logs are saved as JSON files.

In [ ]:
from datetime import datetime
import google.colab.drive
from google.colab import drive

env_str =  "ReinforceTactics"
log_dir = ""
parent_path = ""
use_google_drive = True
if use_google_drive:
    parent_path = "/content/gdrive"
    google.colab.drive.mount(parent_path, force_remount=True)
    log_dir = "{}/MyDrive/Finding Theta/logs/{}".format(parent_path, env_str)
else:
    log_dir = "/content/logs/{}".format(env_str)

tournamnet_log_dir = os.path.join(log_dir, 'tournaments')
time_folder = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
run_folder = os.path.join(tournamnet_log_dir, time_folder)
replay_dir = os.path.join(run_folder, 'replays')
conversation_log_dir = os.path.join(run_folder, 'llm_logs')

#Create Folders
os.makedirs(log_dir, exist_ok=True)
os.makedirs(tournamnet_log_dir, exist_ok=True)
os.makedirs(run_folder, exist_ok=True)
os.makedirs(replay_dir, exist_ok=True)
os.makedirs(conversation_log_dir, exist_ok=True)

print(f"📁 Logs will be saved to: {run_folder}")
print(f"📁 Replay files will be saved to: {replay_dir}")
print(f"📁 Conversation logs will be saved to: {conversation_log_dir}")

In [ ]:
# Run a single game with conversation logging enabled
# Uncomment and run if you have OpenAI API key configured

# if 'openai' in llm_bots_available:
#     import tempfile
#     import logging
#
#     # Enable DEBUG logging to see conversation logs
#     logging.getLogger('reinforcetactics.game.llm_bot').setLevel(logging.DEBUG)
#
#     # Create a temporary directory for logs
#     with tempfile.TemporaryDirectory() as tmpdir:
#         print(f"Conversation logs will be saved to: {tmpdir}")
#
#         winner = run_single_game(
#             player1_bot=llm_bots_available['openai'],
#             player2_bot='simple',
#             map_file='maps/1v1/6x6_beginner.csv',
#             max_turns=100,
#             verbose=True,
#             player1_model='gpt-4o-mini',
#             log_conversations=True,
#             conversation_log_dir=tmpdir
#         )
#
#         # List the log files
#         import os
#         log_files = [f for f in os.listdir(tmpdir) if f.endswith('.json')]
#         print(f"\n📝 Generated {len(log_files)} conversation log files")
#
#         # Show a sample log entry
#         if log_files:
#             import json
#             with open(os.path.join(tmpdir, log_files[0])) as f:
#                 sample_log = json.load(f)
#             print(f"\n📄 Sample log entry:")
#             print(json.dumps(sample_log, indent=2)[:500] + "...")
# else:
#     print("⚠️  OpenAIBot not available. Please install openai and configure API key.")

print("Uncomment the code above to run a game with conversation logging")

### Example 2.6: Reviewing Logged Conversations

Learn how to review and analyze the logged conversations from your games. This example shows how to:
- Use a persistent log directory for Google Colab
- List and inspect log files (now one file per game)
- Extract key information from conversation logs
- Pretty-print log content for analysis

**New in this version:**
- Each game now creates a **single** log file containing all turns
- Filename format: `game_{session_id}_player{N}_model{model}.json`
- Makes it easier to review an entire game's decision-making process


In [ ]:
# Example 2.6: Reviewing Logged Conversations
# Run a game with persistent logging and then review the logs
# Uncomment and run if you have OpenAI API key configured

# if 'openai' in llm_bots_available:
#     import os
#     import json
#     from pathlib import Path
#     import logging
#
#     # Enable DEBUG logging to see conversation logs
#     logging.getLogger('reinforcetactics.game.llm_bot').setLevel(logging.DEBUG)
#
#     # Create persistent log directory (recommended for Google Colab)
#     log_dir = '/content/llm_logs/'
#     os.makedirs(log_dir, exist_ok=True)
#     print(f"📁 Logs will be saved to: {os.path.abspath(log_dir)}\n")
#
#     # Run game with logging
#     print("🎮 Running game with conversation logging...\n")
#     winner = run_single_game(
#         player1_bot=llm_bots_available['openai'],
#         player2_bot='simple',
#         map_file='maps/1v1/6x6_beginner.csv',
#         max_turns=50,  # Shorter game for demo
#         verbose=True,
#         player1_model='gpt-4o-mini',
#         log_conversations=True,
#         conversation_log_dir=log_dir
#     )
#
#     # Helper function to review conversation logs
#     def review_conversation_logs(log_dir):
#         """Review and display game conversation logs."""
#         log_path = Path(log_dir)
#         # NEW: Logs are now stored as game_{session_id}_player{N}_model{model}.json
#         # Each file contains ALL turns from a single game session
#         log_files = sorted(log_path.glob('game_*.json'))
#
#         print(f"\n{'='*70}")
#         print("📊 CONVERSATION LOGS REVIEW")
#         print("="*70)
#         print(f"📁 Absolute path: {log_path.absolute()}")
#         print(f"📝 Total game log files: {len(log_files)}")
#         print(f"💡 Note: Each file now contains ALL turns from one game\n")
#
#         if not log_files:
#             print("⚠️  No log files found. Make sure logging was enabled.")
#             return log_files
#
#         # Show first game log in detail
#         log_file = log_files[0]
#         print(f"\n{'='*70}")
#         print(f"📄 Game log file: {log_file.name}")
#         print(f"📁 Full path: {log_file.absolute()}")
#         print('='*70)
#
#         with open(log_file) as f:
#             log_data = json.load(f)
#
#         # Display game metadata
#         print(f"🎮 Game Session ID: {log_data.get('game_session_id', 'N/A')}")
#         print(f"🤖 Model: {log_data.get('model', 'N/A')}")
#         print(f"🏢 Provider: {log_data.get('provider', 'N/A')}")
#         print(f"👤 Player: {log_data.get('player', 'N/A')}")
#         print(f"🕐 Start Time: {log_data.get('start_time', 'N/A')}")
#         print(f"🔢 Total Turns: {len(log_data.get('turns', []))}")
#
#         # Show sample turns
#         turns = log_data.get('turns', [])
#         if turns:
#             print(f"\n{'='*70}")
#             print("📝 Sample Turns (first and last)")
#             print("="*70)
#
#             for idx in [0, -1] if len(turns) > 1 else [0]:
#                 turn = turns[idx]
#                 print(f"\n🔢 Turn {turn.get('turn_number', 'N/A')}:")
#                 print(f"  🕐 Timestamp: {turn.get('timestamp', 'N/A')}")
#
#                 # Extract reasoning from response
#                 response = turn.get('assistant_response', '')
#                 try:
#                     response_json = json.loads(response)
#                     reasoning = response_json.get('reasoning', 'N/A')
#                     actions = response_json.get('actions', [])
#
#                     print(f"  �� Reasoning: {reasoning[:150]}...")
#                     print(f"  ⚡ Actions: {len(actions)} action(s)")
#
#                     if actions:
#                         action_types = [a.get('type', 'unknown') for a in actions[:3]]
#                         print(f"     First few: {', '.join(action_types)}")
#                 except (json.JSONDecodeError, Exception) as e:
#                     print(f"  Response: {response[:100]}...")
#
#         print(f"\n{'='*70}")
#         print("💡 Tips:")
#         print("  - Each game_*.json file contains ALL turns from that game")
#         print("  - System prompt is stored once at the top of the file")
#         print("  - Use 'jq' for advanced JSON parsing: !jq . path/to/log.json")
#         print("  - Compare reasoning across different games/models")
#         print("="*70 + "\n")
#
#         return log_files
#
#     # Review the logs
#     log_files = review_conversation_logs(log_dir)
#
#     # Optional: Find games by session ID
#     def find_game_log(log_dir, session_id):
#         """Find log file for a specific game session."""
#         log_path = Path(log_dir)
#         for log_file in log_path.glob('game_*.json'):
#             if session_id in log_file.name:
#                 return log_file
#         return None
#
#     print("\n📌 Helper functions defined:")
#     print("  - review_conversation_logs(log_dir): Review game logs")
#     print("  - find_game_log(log_dir, session_id): Find specific game")
# else:
#     print("⚠️  OpenAIBot not available. Please install openai and configure API key.")

print("Uncomment the code above to run Example 2.6")

### Example 3: Mini Tournament

Run a small tournament with available bots.

In [ ]:
# # Define tournament participants
# # Format: (display_name, bot_class_or_'simple', optional_model_name)

# tournament_bots = [
#     ('SimpleBot', 'simple', None),
#     ('MediumBot', 'medium', None),
#     ('AdvancedBot', 'advanced', None),
# ]

# # Add LLM bots if available and configured
# if 'openai' in llm_bots_available and os.environ.get('OPENAI_API_KEY'):
#     tournament_bots.append(('GPT-4o-mini', llm_bots_available['openai'], 'gpt-4o-mini'))

# if 'claude' in llm_bots_available and os.environ.get('ANTHROPIC_API_KEY'):
#     tournament_bots.append(('Claude Haiku', llm_bots_available['claude'], 'claude-3-haiku-20240307'))

# if 'gemini' in llm_bots_available and os.environ.get('GOOGLE_API_KEY'):
#     tournament_bots.append(('Gemini Flash', llm_bots_available['gemini'], 'gemini-1.5-flash'))

# # Run tournament if we have at least 2 bots
# if len(tournament_bots) >= 2:
#     results = run_tournament(
#         bots=tournament_bots,
#         map_file='maps/1v1/beginner.csv',
#         games_per_matchup=2,  # 2 games per side = 4 total per matchup
#         max_turns=100
#     )
#     print("\n✅ Tournament complete! Results saved in 'results' variable.")
# else:
#     print("⚠️  Need at least 2 bots for a tournament.")
#     print("   Configure API keys for LLM bots or add more SimpleBots for testing.")

## 🎨 Custom Model Configuration

You can specify different models for each LLM provider. Here are some examples:

### OpenAI Models

**Available models:**
- `gpt-4o-mini` (default) - Fastest and cheapest, good for testing
- `gpt-4o` - More capable, higher cost
- `gpt-4-turbo` - Previous generation, balanced

**Example:**
```python
# Using GPT-4o for stronger gameplay
winner = run_single_game(
    player1_bot=OpenAIBot,
    player2_bot='simple',
    player1_model='gpt-4o'
)
```

### Claude Models

**Available models:**
- `claude-3-haiku-20240307` (default) - Fastest and cheapest
- `claude-3-5-sonnet-20241022` - Most capable, balanced cost
- `claude-3-opus-20240229` - Highest capability, highest cost

**Example:**
```python
# Using Claude Sonnet for better strategic play
winner = run_single_game(
    player1_bot=ClaudeBot,
    player2_bot='simple',
    player1_model='claude-3-5-sonnet-20241022'
)
```

### Gemini Models

**Available models:**
- `gemini-1.5-flash` (default) - Fast and efficient, free tier available
- `gemini-1.5-pro` - More capable, higher cost
- `gemini-2.0-flash-exp` - Experimental, cutting edge

**Example:**
```python
# Using Gemini Pro for better performance
winner = run_single_game(
    player1_bot=GeminiBot,
    player2_bot='simple',
    player1_model='gemini-1.5-pro'
)
```

### Example: Tournament with Custom Models

Compare different models from different providers:

In [ ]:
# Advanced tournament: Compare different models
# Only run this if you have all API keys configured and don't mind the cost!

# Set logging level
logging.getLogger('reinforcetactics.game.llm_bot').setLevel(logging.CRITICAL)

# Define bots using the new TournamentBot class
advanced_bots = [
    TournamentBot('SimpleBot', 'simple'),
    TournamentBot('MediumBot', 'medium'),
    TournamentBot('AdvancedBot', 'advanced'),
    TournamentBot('Claude Haiku 4.5', ClaudeBot, 'claude-haiku-4-5-20251001', temperature=0.5, max_tokens=None),
    TournamentBot('Claude Sonnet 4.5', ClaudeBot, 'claude-sonnet-4-5-20250929', temperature=0.5, max_tokens=None),
    TournamentBot('Gemini 2.0 Flash', GeminiBot, 'gemini-2.0-flash', temperature=0.5, max_tokens=None),
    TournamentBot('ChatGPT 5 Mini', OpenAIBot, 'gpt-5-mini-2025-08-07', max_tokens=None),
]

maps=['maps/1v1/beginner.csv',
      'maps/1v1/funnel_point.csv',
      'maps/1v1/center_mountains.csv',
      'maps/1v1/corner_points.csv']

results = run_tournament(
    advanced_bots,
    maps=maps,
    games_per_matchup=1,
    map_pool_mode='all',
    max_turns=100,
    should_reason=True,  # Enable reasoning for all LLM bots
    log_conversations=True,
    conversation_log_dir=conversation_log_dir,
    save_replays=True,
    replay_dir=replay_dir
)

print("Uncomment the code above to run a full model comparison tournament")
print("⚠️  Warning: This will make many API calls and may incur costs!")

## 📖 Documentation

In [ ]:
import json
import pandas as pd
import os
from datetime import datetime
from collections import defaultdict
import reinforcetactics

def analyze_token_usage(log_dir):
    """Analyze token usage from log files in the specified directory."""
    if not log_dir or not os.path.exists(log_dir):
        return None

    token_stats = defaultdict(lambda: {'input_tokens': 0, 'output_tokens': 0, 'count': 0})

    # Walk through log directory
    for root, _, files in os.walk(log_dir):
        for file in files:
            if not file.endswith('.json') or not file.startswith('game_'):
                continue

            try:
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    data = json.load(f)

                model = data.get('model', 'unknown')

                # Try to extract usage from turns
                turns = data.get('turns', [])
                file_input_tokens = 0
                file_output_tokens = 0

                for turn in turns:
                    # Check common usage locations (OpenAI, Anthropic, Gemini patterns)
                    usage = turn.get('usage') or turn.get('token_usage') or turn.get('usage_metadata')

                    if usage and isinstance(usage, dict):
                        # Input Tokens
                        file_input_tokens += (usage.get('prompt_tokens') or
                                            usage.get('input_tokens') or
                                            usage.get('prompt_token_count') or 0)
                        # Output Tokens
                        file_output_tokens += (usage.get('completion_tokens') or
                                             usage.get('output_tokens') or
                                             usage.get('candidates_token_count') or 0)

                if file_input_tokens > 0 or file_output_tokens > 0:
                    token_stats[model]['input_tokens'] += file_input_tokens
                    token_stats[model]['output_tokens'] += file_output_tokens
                    token_stats[model]['count'] += 1

            except Exception as e:
                print(f"⚠️ Error reading log file {file}: {e}")

    return dict(token_stats)

def save_tournament_results(results_data, output_dir='tournament_results', llm_log_dir=None):
    """Save tournament results to CSV and JSON files with metadata."""
    if not results_data:
        print("⚠️ No results data to save.")
        return

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Get library version
    rt_version = getattr(reinforcetactics, '__version__', 'unknown')

    # Prepare data with metadata
    # Create a shallow copy to avoid modifying the original dict
    data_to_save = results_data.copy()
    data_to_save['metadata'] = {
        'reinforcetactics_version': rt_version,
        'timestamp': timestamp,
        'export_time': datetime.now().isoformat()
    }

    # Analyze token usage if log directory is provided
    if llm_log_dir:
        print(f"📊 Analyzing token usage from: {llm_log_dir}")
        token_stats = analyze_token_usage(llm_log_dir)
        if token_stats:
            data_to_save['token_stats'] = token_stats
            print("✅ Token usage analysis complete")

            # Create DataFrame for display
            token_data = []
            for model, stats in token_stats.items():
                token_data.append({
                    'Model': model,
                    'Input Tokens': stats['input_tokens'],
                    'Output Tokens': stats['output_tokens'],
                    'Total Tokens': stats['input_tokens'] + stats['output_tokens'],
                    'Games Logged': stats['count']
                })

            if token_data:
                token_df = pd.DataFrame(token_data)
                print("\n💰 Token Usage Summary:")
                display(token_df)

                # Save token stats to CSV
                token_csv_path = os.path.join(output_dir, f'token_usage_{timestamp}.csv')
                token_df.to_csv(token_csv_path, index=False)
                print(f"✅ Token stats saved to: {token_csv_path}")

    # 1. Save full results to JSON
    json_path = os.path.join(output_dir, f'tournament_results_{timestamp}.json')
    try:
        with open(json_path, 'w') as f:
            json.dump(data_to_save, f, indent=2)
        print(f"✅ Full results saved to: {json_path}")
        # print(f"ℹ️  Library Version: {rt_version}")
    except Exception as e:
        print(f"⚠️ Error saving JSON: {e}")

    # 2. Save standings to CSV
    if 'standings' in results_data:
        csv_path = os.path.join(output_dir, f'tournament_standings_{timestamp}.csv')
        try:
            df = pd.DataFrame(results_data['standings'])
            df.to_csv(csv_path, index=False)
            print(f"✅ Standings saved to: {csv_path}")
            print("\nPreview of CSV:")
            display(df)
        except Exception as e:
            print(f"⚠️ Error saving CSV: {e}")

    # 3. Save Matchups to CSV
    if 'matchups' in results_data and results_data['matchups']:
        matchups_csv_path = os.path.join(output_dir, f'tournament_matchups_{timestamp}.csv')
        try:
            matchups_df = pd.DataFrame(results_data['matchups'])
            # Rename columns for better readability
            matchups_df_renamed = matchups_df.rename(columns={
                'bot1': 'Bot 1',
                'bot2': 'Bot 2',
                'bot1_wins': 'Bot 1 Wins',
                'bot2_wins': 'Bot 2 Wins',
                'draws': 'Draws'
            })

            matchups_df_renamed.to_csv(matchups_csv_path, index=False)
            print(f"✅ Matchups saved to: {matchups_csv_path}")
            # print("\nPreview of Matchups CSV:")
            # display(matchups_df_renamed)
        except Exception as e:
            print(f"⚠️ Error saving Matchups CSV: {e}")

        # 4. Save Matrix/Cross Table CSV (NEW)
        matrix_csv_path = os.path.join(output_dir, f'tournament_matrix_{timestamp}.csv')
        try:
            # Extract unique bots
            bots = set()
            for m in results_data['matchups']:
                bots.add(m['bot1'])
                bots.add(m['bot2'])
            bots = sorted(list(bots))

            # Create empty DataFrame
            matrix_df = pd.DataFrame(index=bots, columns=bots)
            # Initialize with "0-0-0" for no games played
            matrix_df = matrix_df.fillna("0-0-0")

            # Fill with W-L-D strings
            for m in results_data['matchups']:
                b1 = m['bot1']
                b2 = m['bot2']
                w1 = int(m['bot1_wins'])
                w2 = int(m['bot2_wins'])
                d = int(m['draws'])

                # Format: Wins-Losses-Draws from the perspective of the row bot
                matrix_df.loc[b1, b2] = f"{w1}-{w2}-{d}"
                matrix_df.loc[b2, b1] = f"{w2}-{w1}-{d}"

            # Mark diagonal
            for b in bots:
                matrix_df.loc[b, b] = 'X'

            matrix_df.to_csv(matrix_csv_path)
            print(f"✅ Matrix table saved to: {matrix_csv_path}")
            print("\nPreview of Tournament Matrix (W-L-D):")
            display(matrix_df)
        except Exception as e:
             print(f"⚠️ Error saving Matrix CSV: {e}")

# Save the results from the previous tournament
if 'results' in locals():
    # Use run_folder if defined, otherwise default
    out_dir = run_folder if 'run_folder' in locals() else 'tournament_results'
    # Use conversation_log_dir if defined
    llm_log_dir = conversation_log_dir if 'conversation_log_dir' in locals() else None

    save_tournament_results(results, out_dir, llm_log_dir=llm_log_dir)
else:
    print("⚠️ 'results' variable not found. Run a tournament first!")

### 💰 API Cost Estimates

**OpenAI Pricing (approximate, as of 2024):**
- GPT-4o-mini: $0.15/1M input tokens, $0.60/1M output tokens
- GPT-4o: $2.50/1M input tokens, $10.00/1M output tokens
- Typical game: 2,000-10,000 tokens per side
- **Cost per game:** $0.0001-0.0005 (mini), $0.005-0.02 (4o)

**Anthropic Pricing:**
- Claude Haiku: $0.25/1M input tokens, $1.25/1M output tokens
- Claude Sonnet: $3.00/1M input tokens, $15.00/1M output tokens
- **Cost per game:** $0.0001-0.0003 (Haiku), $0.003-0.015 (Sonnet)

**Google Gemini Pricing:**
- Gemini Flash: Free tier available (15 RPM), $0.075/1M input, $0.30/1M output
- Gemini Pro: $1.25/1M input tokens, $5.00/1M output tokens
- **Cost per game:** ~$0 (Flash free tier), $0.0001-0.001 (Flash paid), $0.001-0.005 (Pro)

**Tournament Cost Estimates:**
- 3 bots, 2 games/matchup: 12 games total
- Using mini/haiku/flash: ~$0.001-0.01 total
- Using premium models: ~$0.05-0.20 total

**Cost Saving Tips:**
1. Use mini/haiku/flash models for development and testing
2. Use smaller maps (6x6) which need fewer tokens
3. Set lower `max_turns` to prevent long games
4. Use Gemini Flash free tier for unlimited testing

### ⚡ Performance Tips

**Model Selection for Different Purposes:**

**For Testing & Development:**
- ✅ GPT-4o-mini - Fast, cheap, decent strategy
- ✅ Claude Haiku - Very fast, cheap, good baseline
- ✅ Gemini Flash - Free tier, fast, great for testing

**For Competitive Play:**
- 🏆 GPT-4o - Strong strategic thinking
- 🏆 Claude Sonnet 3.5 - Excellent reasoning, good value
- 🏆 Gemini Pro 1.5 - Good balance of cost/performance

**For Research/Analysis:**
- 🔬 Claude Opus - Highest reasoning capability
- 🔬 GPT-4 Turbo - Consistent performance

**Game Speed:**
- Smaller maps (6x6) complete in 1-3 minutes per game
- Larger maps (32x32) can take 10-30 minutes per game
- LLM API calls add 1-5 seconds per turn
- SimpleBot is nearly instant

**Tournament Duration Estimates:**
- 2 bots, 4 games: ~5-15 minutes
- 3 bots, 12 games: ~15-45 minutes
- 4 bots, 24 games: ~30-90 minutes
- 5 bots, 40 games: ~1-2 hours

**Optimization Strategies:**
1. Start with 6x6 maps for quick iterations
2. Use `games_per_matchup=1` for initial testing
3. Set `max_turns=100` for faster games
4. Run tournaments with fewer bots initially
5. Use verbose=False in run_single_game() to reduce output

### 🔧 Troubleshooting

**Problem: "API key not provided" error**
- **Solution:** Make sure you've set the API key in the environment or Colab secrets
- Check the API Key Configuration section output
- Uncomment and set the API key directly in the configuration cell

**Problem: "openai package not installed" error**
- **Solution:** Run the installation cell again
- Or manually install: `!pip install openai>=1.0.0`

**Problem: Bot makes invalid moves or errors**
- **Solution:** This is expected occasionally with LLMs
- The game will skip invalid actions and continue
- Try using a more capable model (e.g., GPT-4o instead of mini)
- Check the logs to see what actions failed

**Problem: Games taking too long**
- **Solution:** Reduce `max_turns` parameter
- Use smaller maps (6x6 instead of 32x32)
- Faster models: Haiku, Flash, or mini

**Problem: API rate limits exceeded**
- **Solution:** Add delays between games
- Use free tier models (Gemini Flash)
- Reduce `games_per_matchup`
- Spread tournament over multiple sessions

**Problem: Out of memory error**
- **Solution:** Restart the notebook runtime
- Run fewer games at once
- Use smaller maps

**Problem: "Map file not found" error**
- **Solution:** Make sure you're in the reinforce-tactics directory
- Check the path: `!ls maps/1v1/`
- Use absolute paths if needed

**Problem: Import errors for LLM bots**
- **Solution:** Check that dependencies installed correctly
- Verify the repository was cloned successfully
- Make sure the repository is in your Python path

**Problem: Tournament results seem random**
- **Solution:** LLMs have inherent randomness
- Increase `games_per_matchup` for more stable results
- Temperature parameter affects consistency (set in llm_bot.py)
- SimpleBot is deterministic and provides a good baseline

**Problem: Cost concerns**
- **Solution:** Always use mini/haiku/flash for testing
- Monitor API usage in your provider dashboard
- Set spending limits in your API account
- Test with SimpleBot first (free)
- Use Gemini Flash free tier for unlimited testing

## 🎓 Next Steps

**New Features Now Available:**
1. ✅ Elo rating system - Track bot skill across games
2. ✅ Multiple maps support - Run tournaments across different maps
3. ✅ Per-map statistics - See which bots excel on specific maps

**Experiment Ideas:**
1. Compare different models from the same provider
2. Test how map size affects bot performance using multiple maps
3. Analyze which bots excel at different strategies
4. Track Elo rating progression over multiple tournaments
5. Compare performance across maps of different sizes

**Code Customization:**
1. Modify system prompts in `llm_bot.py` for different strategies
2. Add logging to track specific metrics
3. Create visualization of tournament brackets
4. Export results to CSV for analysis
5. Build a web interface for live tournaments

**Advanced Tournaments:**
1. Swiss-system tournament format
2. Double elimination brackets
3. Time-limited games
4. Asymmetric maps
5. Team battles (coming soon)

**Contributing:**
- Found a bug? Open an issue on GitHub
- Have an improvement? Submit a pull request
- Share your tournament results!

**Resources:**
- Repository: https://github.com/kuds/reinforce-tactics
- Game Rules: See `reinforcetactics/game/llm_bot.py` SYSTEM_PROMPT
- Tournament Script: `scripts/tournament.py`

---

**Happy Gaming! 🎮**

In [ ]:
# Create version execution
notebook_name = "notebook.ipynb"
%notebook -e $notebook_name

In [ ]:
import os
import shutil
source_file = os.path.join(notebook_name)
destination_file = os.path.join(run_folder, notebook_name)

try:
    shutil.copyfile(notebook_name, destination_file)
    print(f"File '{source_file}' copied to '{destination_file}' successfully.")
except FileNotFoundError:
    print(f"Error: Source file '{source_file}' not found.")
except shutil.SameFileError:
    print(f"Error: Source and destination are the same file.")
except Exception as e:
    print(f"An error occurred: {e}")